In [ ]:
import os
import pandas as pd
import csv
import numpy as np

bucket = os.getenv('WORKSPACE_BUCKET')
demo = pd.read_csv(f'{bucket}/data/gwas_v2/phenotypes/celiac_demo_v2.tsv',sep='\t')
match = pd.read_csv(f'{bucket}/data/gwas_v2/phenotypes/celiac_matched_data_v2.tsv',sep='\t')

In [ ]:
#uncomment this cell and run in a dataproc cluster only if you do not have genetic sex computed already; this can take 2 hours
"""import os
bucket = os.getenv("WORKSPACE_BUCKET")
import hail as hl
hl.init(default_reference='GRCh38', idempotent=True)


table = hl.read_matrix_table(os.getenv("WGS_ACAF_THRESHOLD_SPLIT_HAIL_PATH"))
imputed_sex = hl.impute_sex(table.GT,female_threshold=0.35, male_threshold=0.75)
female = imputed_sex.filter(imputed_sex.is_female == True)
male = imputed_sex.filter(imputed_sex.is_female == False)
females_genetic = female.s.collect()
males_genetic = male.s.collect()

pd.Series(males_genetic).to_csv(f'{bucket}/data/gen_males.csv',index=False)
pd.Series(females_genetic).to_csv(f'{bucket}/data/gen_females.csv',index=False)"""

In [ ]:
females_genetic=pd.read_csv(f'{bucket}/data/gen_females.csv')

In [ ]:
relatedness=pd.read_csv(f'{bucket}/data/relatedness_flagged_samples.tsv')
match=match[~match['person_id'].isin(relatedness['sample_id'])]

In [ ]:
#count demographics for tables
pd.DataFrame(match.value_counts(['sex_at_birth','label'])).reset_index().pivot(columns='label',values='count',index='sex_at_birth')

In [ ]:
match.groupby('label').agg({'age':['mean','std']})

In [ ]:
#ttest for age
from scipy.stats import ttest_ind
result=ttest_ind(match.loc[match['label']==1,'age'],match.loc[match['label']==0,'age'])
print(result)

In [ ]:
match.loc[match['person_id'].isin(females_genetic['0']),'is_female']=1
match.is_female.fillna(0,inplace=True)

In [ ]:
pheno=match[['person_id','label','age','PC1','PC2','PC3','is_female']]


In [ ]:
pheno.to_csv(f'{bucket}/data/gwas_v4/pheno_hail_final.tsv',sep='\t',index=False)

In [ ]:
pheno=pd.read_csv(f'{bucket}/data/gwas_v4/pheno_hail_final.tsv',sep='\t')